In [ ]:
from labda_parsers import Sens
from acti_motus import Features

from pyinstrument import Profiler

import logging

# Configure logging to see info messages
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

df = Sens().from_file('../test_data/thigh.bin')

df.iloc[500:5000] = 0
df.iloc[20000:90000] = 0
df.iloc[150000:200000] = 0

profiler = Profiler()

with profiler:
    features = Features()
    extracted = features.extract(df)
    # timestamps, data = features._df_to_server(df)
    # sens = features.sens_extract(timestamps, data)

extracted.to_parquet('../temp/features.parquet')
profiler.write_html('../temp/features.html')

In [ ]:
from acti_motus.classification.thigh import Thigh
import pandas as pd
import logging

# Configure logging to see info messages
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

df = pd.read_parquet('../temp/features.parquet')
sensor = Thigh(vendor='Sens')

df[['inclination', 'side_tilt', 'direction']] = sensor.get_angles(df)
df['non_wear'] = sensor.get_non_wear(df)

sensor.check_inside_out_flip(df), sensor.check_upside_down_flip(df)